In [1]:
import polars as pl
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import pandas as pd
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
from seqeval.metrics import classification_report, f1_score, accuracy_score
from sklearn.metrics import accuracy_score as sk_accuracy_score, classification_report as sk_classification_report


In [2]:
label2id = {"O": 0, "B-ASP": 1, "I-ASP": 2}
id2label = {v: k for k, v in label2id.items()}
sentiment2id = {
    "negative": 0,
    "positive": 1,
    "neutral": 2
}
id2sentiment = {v: k for k, v in sentiment2id.items()}

In [3]:
df=pl.read_parquet("../data/processed/df_aspect_pos.parquet")

In [4]:
df.columns

['input_ids',
 'attention_mask',
 'labels',
 'aspects_index',
 'aspects_sentiment',
 'type']

In [5]:
def custom_collate_fn(batch):
    input_ids = torch.stack([item["input_ids"] for item in batch])
    attention_mask = torch.stack([item["attention_mask"] for item in batch])
    labels = torch.stack([item["labels"] for item in batch])

    aspects_index = [item["aspects_index"] for item in batch]          # List[List[List[int]]]
    aspects_sentiment = [item["aspects_sentiment"] for item in batch]  # List[List[int]]

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
        "aspects_index": aspects_index,
        "aspects_sentiment": aspects_sentiment,
    }


In [6]:
def extract_aspect_spans(pred_labels):
    spans = []
    i = 0
    while i < len(pred_labels):
        if pred_labels[i] == 1:  # B-ASP
            start = i
            i += 1
            while i < len(pred_labels) and pred_labels[i] == 2:  # I-ASP
                i += 1
            end = i - 1
            spans.append([start, end])
        else:
            i += 1
    return spans

In [7]:
class CustomDataset(Dataset):
    def __init__(self, df):
        self.input_ids = df["input_ids"].to_numpy()
        self.attention_mask = df["attention_mask"].to_numpy()
        self.labels = df["labels"].to_numpy()
        self.aspects_index = df["aspects_index"].to_list()
        self.aspects_sentiment = df["aspects_sentiment"].to_list()

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": torch.tensor(self.input_ids[idx], dtype=torch.long),
            "attention_mask": torch.tensor(self.attention_mask[idx], dtype=torch.long),
            "labels": torch.tensor(self.labels[idx], dtype=torch.long),
            "aspects_index": self.aspects_index[idx],           # list of [start, end]
            "aspects_sentiment": self.aspects_sentiment[idx],   # list of sentiment values
        }


batch_size = 32
train_dataset = CustomDataset(df.filter(pl.col("type") != "val"))
val_dataset = CustomDataset(df.filter(pl.col("type") == "val"))
test_dataset = CustomDataset(df.filter(pl.col("type") == "test"))
val_dataloader = DataLoader(val_dataset, batch_size=batch_size , collate_fn=custom_collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=custom_collate_fn)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn)


In [8]:
for batch in train_dataloader:
    print(batch["input_ids"].shape)
    print(batch["attention_mask"].shape)
    print(batch["labels"].shape)
    print(batch["aspects_sentiment"])
    print(batch["aspects_index"])
    break

torch.Size([32, 128])
torch.Size([32, 128])
torch.Size([32, 128])
[[1, 1, 1], [0], [1, 2, 2, 1], [0, 1, 0, 2, 2], [1, 1], [1], [1], [0], [1], [0, 0, 0], [1], [1], [1], [1], [0], [0], [0], [0], [1], [1], [1], [1], [1], [1, 1], [1, 0], [1, 0], [1, 1], [1], [1], [0], [1], [1]]
[[[2, 2], [5, 5], [8, 8]], [[1, 1]], [[1, 1], [8, 8], [23, 23], [19, 20]], [[15, 16], [10, 10], [22, 22], [29, 29], [35, 35]], [[3, 5], [33, 35]], [[9, 9]], [[13, 13]], [[6, 9]], [[6, 8]], [[7, 9], [11, 14], [16, 19]], [[7, 7]], [[10, 10]], [[2, 2]], [[2, 2]], [[2, 2]], [[7, 7]], [[1, 2]], [[1, 2]], [[3, 4]], [[2, 2]], [[10, 14]], [[7, 12]], [[5, 5]], [[5, 6], [12, 12]], [[4, 5], [9, 9]], [[3, 3], [15, 15]], [[21, 21], [4, 5]], [[5, 5]], [[3, 3]], [[3, 4]], [[8, 8]], [[3, 4]]]


In [9]:
class AspectDetectionModel(nn.Module):
    def __init__(self):
        super(AspectDetectionModel, self).__init__()
        self.bert = AutoModel.from_pretrained("bert-base-uncased")
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(self.bert.config.hidden_size, len(label2id))
        self.sentiment_classifier = nn.Linear(self.bert.config.hidden_size, 3)  # positive, negative, neutral

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = self.dropout(outputs.last_hidden_state)  # [B, L, H]
        logits = self.classifier(sequence_output)  # [B, L, num_labels]
        return logits, sequence_output

model = AspectDetectionModel().to("mps")
criterion = nn.CrossEntropyLoss(ignore_index=-100)
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
optimizer = optimizer = torch.optim.AdamW([
    {'params': model.bert.parameters(), 'lr': 2e-5},
    {'params': model.classifier.parameters(), 'lr': 5e-5},
    {'params': model.sentiment_classifier.parameters(), 'lr': 5e-5}
])
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.8)

In [10]:
num_epochs = 10
true_tags = []
pred_tags = []
true_sentiments = []
pred_sentiments = []

for epoch in range(num_epochs):
    total_aspect_train_loss = 0
    total_sentiment_train_loss = 0
    total_aspect_val_loss = 0
    total_sentiment_val_loss = 0

    model.train()
    total_train_loss = 0

    for batch in train_dataloader:
        input_ids = batch["input_ids"].to("mps")
        attention_mask = batch["attention_mask"].to("mps")
        labels = batch["labels"].to("mps")

        # logits contains the aspect extraction head output and sequence_output contains the contextualized embeddings
        logits, sequence_output = model(input_ids, attention_mask)
        aspect_loss = criterion(logits.view(-1, len(label2id)), labels.view(-1))
        total_aspect_train_loss += aspect_loss.item()

        sentiment_losses = []
        for i in range(len(input_ids)):
            for aspect_index, sentiment in zip(batch["aspects_index"][i], batch["aspects_sentiment"][i]):
                if aspect_index[1] >= sequence_output.size(1):
                    continue
                # Assume the aspect span is a list of words ['chrome', '##book'], BIO tags are ['B-ASP', 'I-ASP'], and indices are [15, 16]
                # Each word in the aspect span have its own embedding in the sequence output
                # We take the mean of the embeddings for the aspect span
                # aspect_index = [15, 16] means we take the mean of sequence_output[i, 15:17]
                pooled = sequence_output[i, aspect_index[0]:aspect_index[1]+1].mean(dim=0)
                sentiment_logits = model.sentiment_classifier(pooled.unsqueeze(0))
                sentiment_target = torch.tensor([sentiment], dtype=torch.long).to("mps")
                sentiment_loss = criterion(sentiment_logits.view(-1, 3), sentiment_target)
                sentiment_losses.append(sentiment_loss)


        if sentiment_losses:
            sentiment_loss = torch.stack(sentiment_losses).mean()
        else:
            sentiment_loss = torch.tensor(0.0).to("mps")

        total_sentiment_train_loss += sentiment_loss.item()
        total_loss = aspect_loss + sentiment_loss

        total_loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    with torch.no_grad():
        model.eval()

        for batch in val_dataloader:
            input_ids = batch["input_ids"].to("mps")
            attention_mask = batch["attention_mask"].to("mps")
            labels = batch["labels"].to("mps")

            logits, sequence_output = model(input_ids, attention_mask)
            aspect_loss = criterion(logits.view(-1, len(label2id)), labels.view(-1))
            total_aspect_val_loss += aspect_loss.item()

            sentiment_losses = []
            preds = torch.argmax(logits, dim=2)
            preds = preds.detach().cpu().numpy()

            for true, pred in zip(labels.cpu().numpy(), preds):
                filtered_true = []
                filtered_pred = []
                for t, p in zip(true, pred):
                    if t != -100:
                        filtered_true.append(id2label[t])
                        filtered_pred.append(id2label[p])
                true_tags.append(filtered_true)
                pred_tags.append(filtered_pred)

            for i in range(len(input_ids)):
                # During training, I used the existing aspect spans from the training data. But during evaluation, 
                # I will extract the aspect spans from the predicted labels
                # This is because the aspect spans are not available in the validation data
                # I will use the predicted labels to extract the aspect spans
                # The predicted labels are in the form of BIO tags 
                aspects = extract_aspect_spans(preds[i])
                for aspect_index, sentiment in zip(batch["aspects_index"][i], batch["aspects_sentiment"][i]):
                    if aspect_index in aspects and aspect_index[1] < sequence_output.size(1):
                        pooled = sequence_output[i, aspect_index[0]:aspect_index[1]+1].mean(dim=0)
                        sentiment_logits = model.sentiment_classifier(pooled.unsqueeze(0))
                        sentiment_target = torch.tensor([sentiment], dtype=torch.long).to("mps")
                        sentiment_loss = criterion(sentiment_logits.view(-1, 3), sentiment_target)
                        sentiment_losses.append(sentiment_loss)
                        pred_sentiments.append(sentiment_logits.argmax(dim=-1).item())
                        true_sentiments.append(sentiment)

            if sentiment_losses:
                sentiment_loss = torch.stack(sentiment_losses).mean()
            else:
                sentiment_loss = torch.tensor(0.0).to("mps")

            total_sentiment_val_loss += sentiment_loss.item()

    scheduler.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], "
        f"Train Aspect Loss: {total_aspect_train_loss/len(train_dataloader):.4f}, "
        f"Train Sentiment Loss: {total_sentiment_train_loss/len(train_dataloader):.4f}, "
        f"Val Aspect Loss: {total_aspect_val_loss/len(val_dataloader):.4f}, "
        f"Val Sentiment Loss: {total_sentiment_val_loss/len(val_dataloader):.4f}")

print("Aspect Extraction Metrics:")
print(classification_report(true_tags, pred_tags))
print("Aspect F1:", f1_score(true_tags, pred_tags))

print("Sentiment Classification Metrics:")
print(sk_classification_report(true_sentiments, pred_sentiments))
print("Sentiment Accuracy:", sk_accuracy_score(true_sentiments, pred_sentiments))

Epoch [1/10], Train Aspect Loss: 0.2662, Train Sentiment Loss: 0.4474, Val Aspect Loss: 0.1475, Val Sentiment Loss: 0.2836
Epoch [2/10], Train Aspect Loss: 0.1279, Train Sentiment Loss: 0.2362, Val Aspect Loss: 0.1201, Val Sentiment Loss: 0.2555
Epoch [3/10], Train Aspect Loss: 0.1013, Train Sentiment Loss: 0.1403, Val Aspect Loss: 0.1201, Val Sentiment Loss: 0.3346
Epoch [4/10], Train Aspect Loss: 0.0857, Train Sentiment Loss: 0.0852, Val Aspect Loss: 0.1247, Val Sentiment Loss: 0.4020
Epoch [5/10], Train Aspect Loss: 0.0739, Train Sentiment Loss: 0.0503, Val Aspect Loss: 0.1210, Val Sentiment Loss: 0.3956
Epoch [6/10], Train Aspect Loss: 0.0638, Train Sentiment Loss: 0.0377, Val Aspect Loss: 0.1159, Val Sentiment Loss: 0.3694
Epoch [7/10], Train Aspect Loss: 0.0550, Train Sentiment Loss: 0.0262, Val Aspect Loss: 0.1210, Val Sentiment Loss: 0.3920
Epoch [8/10], Train Aspect Loss: 0.0477, Train Sentiment Loss: 0.0193, Val Aspect Loss: 0.1294, Val Sentiment Loss: 0.4170
Epoch [9/10], Tr

In [11]:
import os

name="AspectDetectionModel_Sentiment_Analysis"

os.makedirs("../models/AspectDetectionModel", exist_ok=True)

torch.save(model.state_dict(), "../models/AspectDetectionModel/" + name + ".pth")

model_config = {
	"hidden_size": model.bert.config.hidden_size,
	"num_labels": len(label2id),
	"id2label": id2label,
	"label2id": label2id,
	"name": name
}

import json
with open(f"../models/AspectDetectionModel/{name}_config.json", "w") as f:
	json.dump(model_config, f)

print("Model saved successfully to ../models/AspectDetectionModel/")

Model saved successfully to ../models/AspectDetectionModel/


In [12]:
name="AspectDetectionModel_Sentiment_Analysis"
model = AspectDetectionModel()
model.load_state_dict(torch.load("../models/AspectDetectionModel/" + name + ".pth"))
model = model.to("mps")
model.eval()

AspectDetectionModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

In [20]:
temp = tokenizer(("""the high prices you ' re going to pay is for the view not for the food .""").split(), is_split_into_words=True,
                          truncation=True,
                          padding="max_length",
                          max_length=128)

print(temp)

{'input_ids': [101, 1996, 2152, 7597, 2017, 1005, 2128, 2183, 2000, 3477, 2003, 2005, 1996, 3193, 2025, 2005, 1996, 2833, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [28]:
temp = tokenizer(("""This remote control car is fun, fast, and easy to handle—perfect for kids! The build quality is sturdy and it runs smoothly on different surfaces. Battery life is decent and the controls are very responsive. A great gift for kids!""").split(), is_split_into_words=True,
                          truncation=True,
                          padding="max_length",
                          max_length=128)

print(temp)

{'input_ids': [101, 2023, 6556, 2491, 2482, 2003, 4569, 1010, 3435, 1010, 1998, 3733, 2000, 5047, 1517, 3819, 2005, 4268, 999, 1996, 3857, 3737, 2003, 23073, 1998, 2009, 3216, 15299, 2006, 2367, 9972, 1012, 6046, 2166, 2003, 11519, 1998, 1996, 7711, 2024, 2200, 26651, 1012, 1037, 2307, 5592, 2005, 4268, 999, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [36]:
temp = tokenizer(("""Car quality is very nice but the controller sucks . The controller of this car do not works properly and the final in the controller do not rotate fully it only rotate like button""").split(), is_split_into_words=True,
                          truncation=True,
                          padding="max_length",
                          max_length=128)

print(temp)

{'input_ids': [101, 2482, 3737, 2003, 2200, 3835, 2021, 1996, 11486, 19237, 1012, 1996, 11486, 1997, 2023, 2482, 2079, 2025, 2573, 7919, 1998, 1996, 2345, 1999, 1996, 11486, 2079, 2025, 24357, 3929, 2009, 2069, 24357, 2066, 6462, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [37]:
device="mps"
model.eval()
with torch.no_grad():

    
    input_ids = torch.tensor(temp["input_ids"], dtype=torch.long).unsqueeze(0).to(device)
    attention_mask = torch.tensor(temp["attention_mask"], dtype=torch.long).unsqueeze(0).to(device)

    logits, sequence_output = model(input_ids, attention_mask)
    # decode the logits to get the predicted labels
    preds = torch.argmax(logits, dim=2)[0]
    aspects = extract_aspect_spans(preds.cpu().tolist())
    sentiments=[]


    for aspect in aspects:
        pooled = sequence_output[0, aspect[0]:aspect[1]+1].mean(dim=0)
        sentiment_logits = model.sentiment_classifier(
            pooled.unsqueeze(0)
        )
        sentiments.append({"pos":aspect,"senti":torch.argmax(sentiment_logits, dim=1).item()})

preds

tensor([0, 1, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 2, 0, 0,
        0, 0, 0, 0, 0, 1, 1, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='mps:0')

In [38]:
logits

tensor([[[ 3.8200e+00, -5.2799e-01, -3.2160e+00],
         [ 9.7274e-01,  2.4056e+00, -3.9802e+00],
         [ 4.5306e-01, -2.9232e+00,  2.0572e+00],
         [ 5.9108e+00, -2.5656e+00, -3.7683e+00],
         [ 6.4181e+00, -2.5217e+00, -4.2056e+00],
         [ 5.9672e+00, -2.2473e+00, -4.2224e+00],
         [ 7.1839e+00, -3.1137e+00, -4.0946e+00],
         [ 6.1606e+00, -2.8793e+00, -3.6381e+00],
         [-7.0848e-01,  3.4834e+00, -2.2773e+00],
         [ 6.2572e+00, -3.4404e+00, -3.1373e+00],
         [ 6.5763e+00, -2.8340e+00, -3.6271e+00],
         [ 5.6258e+00, -2.7777e+00, -3.3384e+00],
         [-9.3669e-01,  3.0684e+00, -1.7020e+00],
         [ 3.0925e+00, -3.8961e+00,  1.7643e-01],
         [ 5.8309e+00, -3.6962e+00, -3.1830e+00],
         [ 3.4539e+00, -1.1128e+00, -2.3963e+00],
         [ 6.1784e+00, -3.7505e+00, -2.9349e+00],
         [ 6.3078e+00, -3.0557e+00, -3.3462e+00],
         [ 6.3697e+00, -3.6102e+00, -3.3413e+00],
         [ 6.4227e+00, -3.1592e+00, -3.6249e+00],


In [39]:
arr=[-1] * 128
arr

for item in sentiments:
    pos=item["pos"]
    start = pos[0]
    end = pos[0]+pos[1]
    arr[start:end] = [item["senti"]] * (end-start)

In [40]:
aspects

[[1, 2],
 [8, 8],
 [12, 12],
 [25, 25],
 [34, 34],
 [50, 50],
 [51, 53],
 [66, 66],
 [77, 77],
 [78, 78],
 [95, 95],
 [107, 107]]

In [41]:
predictions = [id2label[label] for label in preds.tolist()]
predictions

['O',
 'B-ASP',
 'I-ASP',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ASP',
 'O',
 'O',
 'O',
 'B-ASP',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ASP',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ASP',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ASP',
 'B-ASP',
 'I-ASP',
 'I-ASP',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ASP',
 'O',
 'I-ASP',
 'I-ASP',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ASP',
 'B-ASP',
 'O',
 'I-ASP',
 'I-ASP',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ASP',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ASP',
 'O',
 'I-ASP',
 'I-ASP',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [42]:
def print_sentiment(bio_tag, sentiment):
    if(bio_tag in ["B-ASP", "I-ASP"]):
        return id2sentiment[sentiment]
    else:
        return ""

rows = []

for item in zip(predictions, temp["input_ids"], arr):
    word = tokenizer.convert_ids_to_tokens(item[1])
    if word in ['[CLS]', '[SEP]', '[PAD]']:
        continue
    sentiment = print_sentiment(item[0], item[2])
    rows.append({
        "BIO Tag": item[0],
        "Word": word,
        "Sentiment": sentiment
    })

df = pd.DataFrame(rows)
df

,BIO Tag,Word,Sentiment
0,B-ASP,car,positive
1,I-ASP,quality,positive
2,O,is,
3,O,very,
4,O,nice,
5,O,but,
6,O,the,
7,B-ASP,controller,negative
8,O,sucks,
9,O,.,


In [43]:
df.to_html(index=False).replace("\n","")

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th>BIO Tag</th>      <th>Word</th>      <th>Sentiment</th>    </tr>  </thead>  <tbody>    <tr>      <td>B-ASP</td>      <td>car</td>      <td>positive</td>    </tr>    <tr>      <td>I-ASP</td>      <td>quality</td>      <td>positive</td>    </tr>    <tr>      <td>O</td>      <td>is</td>      <td></td>    </tr>    <tr>      <td>O</td>      <td>very</td>      <td></td>    </tr>    <tr>      <td>O</td>      <td>nice</td>      <td></td>    </tr>    <tr>      <td>O</td>      <td>but</td>      <td></td>    </tr>    <tr>      <td>O</td>      <td>the</td>      <td></td>    </tr>    <tr>      <td>B-ASP</td>      <td>controller</td>      <td>negative</td>    </tr>    <tr>      <td>O</td>      <td>sucks</td>      <td></td>    </tr>    <tr>      <td>O</td>      <td>.</td>      <td></td>    </tr>    <tr>      <td>O</td>      <td>the</td>      <td></td>    </tr>    <tr>      <td>B-ASP</td>      <td>controller</td

In [ ]:
def extract_aspects(input_ids, predictions, tokenizer):
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    
    aspects = []
    current_aspect = []
    current_position = []
    current_idx = 0

    for idx, (token, label) in enumerate(zip(tokens, predictions[0])):
        if token in [tokenizer.cls_token, tokenizer.sep_token, tokenizer.pad_token]:
            continue
            
        label_tag = id2label.get(label, "O")
        
        if label_tag == "B-ASP":
            if current_aspect:
                aspects.append((" ".join(current_aspect), (current_position[0], current_position[-1])))
            current_aspect = [token]
            current_position = [idx]
        elif label_tag == "I-ASP" and current_aspect:
            current_aspect.append(token)
            current_position.append(idx)
        else:
            if current_aspect:
                aspects.append((" ".join(current_aspect), (current_position[0], current_position[-1])))
                current_aspect = []
                current_position = []

    if current_aspect:
        aspects.append((" ".join(current_aspect), (current_position[0], current_position[-1])))

    clean_aspects = []
    for aspect, pos in aspects:
        cleaned = ""
        for word in aspect.split():
            if word.startswith("##"):
                cleaned += word[2:]  # Remove ## prefix
            else:
                if cleaned:
                    cleaned += " "
                cleaned += word
        clean_aspects.append((cleaned, pos))

    return clean_aspects


extracted_aspects = extract_aspects(input_ids, predictions, tokenizer)
extracted_aspects

[('chrome', (2, 2)), ('book', (3, 3))]